In [405]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_absolute_error


Plan:
 - Load sets
 - Clean:
    - remove duplicate entries from sets
    - Drop the unimportant coloumns that we found
    - Make querters into hours
    - remove values from train_targets that are nan in both x and y sets
    - make time features
    - drop datetime coloumns
 - Set prep:
    - 75/25 fordeling. 75% av OBSERVED er trening, resterende 25% er validation, 75% av ESTIMATED er trening, resterende 25% er validation
- Concat the different locations into 1 set, where location is a feature as well
   - 1 hot encoding
- Make a time series model

#### Loading all datasets

In [406]:
#A
y_a = pd.read_parquet('../data/A/train_targets.parquet')
X_test_estimated_a = pd.read_parquet('../data/A/X_test_estimated.parquet')
X_train_estimated_a = pd.read_parquet('../data/A/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('../data/A/X_train_observed.parquet')

# B
y_b = pd.read_parquet('../data/B/train_targets.parquet')
X_test_estimated_b = pd.read_parquet('../data/B/X_test_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../data/B/X_train_estimated.parquet')
X_train_observed_b = pd.read_parquet('../data/B/X_train_observed.parquet')

# C
y_c = pd.read_parquet('../data/C/train_targets.parquet')
X_test_estimated_c = pd.read_parquet('../data/C/X_test_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../data/C/X_train_estimated.parquet')
X_train_observed_c = pd.read_parquet('../data/C/X_train_observed.parquet')

#### Removing duplicate entries from the sets if any exists

In [407]:
# Function that removes the duplicates if it finds duplicates in the specified coloumn
def remove_duplicates_in_coloumn(df, col):
    duplicate_mask = df[col].duplicated(keep="first")
    if duplicate_mask.any():
        df = df[~duplicate_mask]
    return df

In [408]:
#A
y_a = remove_duplicates_in_coloumn(y_a, "time")
X_test_estimated_a = remove_duplicates_in_coloumn(X_test_estimated_a, "date_forecast")
X_train_estimated_a = remove_duplicates_in_coloumn(X_train_estimated_a, "date_forecast")
X_train_observed_a = remove_duplicates_in_coloumn(X_train_observed_a, "date_forecast")

#B
y_b = remove_duplicates_in_coloumn(y_b, "time")
X_test_estimated_b = remove_duplicates_in_coloumn(X_test_estimated_b, "date_forecast")
X_train_estimated_b = remove_duplicates_in_coloumn(X_train_estimated_b, "date_forecast")
X_train_observed_b = remove_duplicates_in_coloumn(X_train_observed_b, "date_forecast")

#C
y_c = remove_duplicates_in_coloumn(y_c, "time")
X_test_estimated_c = remove_duplicates_in_coloumn(X_test_estimated_c, "date_forecast")
X_train_estimated_c = remove_duplicates_in_coloumn(X_train_estimated_c, "date_forecast")
X_train_observed_c = remove_duplicates_in_coloumn(X_train_observed_c, "date_forecast")


In [409]:
#list of all estimated and observed sets

list_of_all_estimated_and_observed_sets = [X_test_estimated_a, X_train_estimated_a, X_train_observed_a,
                                           X_test_estimated_b, X_train_estimated_b, X_train_observed_b,
                                           X_test_estimated_c, X_train_estimated_c, X_train_observed_c]

#### Dropping some coloumns for the bants

In [410]:
# for set in list_of_all_estimated_and_observed_sets:
#     # set.drop("snow_density:kgm3", axis=1, inplace=True)
#     # these 2 had a lot of NaN values
#     set.drop("ceiling_height_agl:m", axis=1, inplace=True)
#     set.drop("cloud_base_agl:m", axis=1,inplace=True) # could potentially not drop this, but set all nan values to 0

#### Converting every 4 quarters into an whole hour

In [411]:
## It might be better not to resample actually, but just use the actual hours that correspond perfectly

def convert_df_into_hourly(df):
    df.set_index("date_forecast", inplace=True)
    df = df.resample('1H').mean()
    df.reset_index(inplace=True)
    return df

In [412]:

X_train_estimated_a = convert_df_into_hourly(X_train_estimated_a)
X_train_observed_a = convert_df_into_hourly(X_train_observed_a)
X_test_estimated_a = convert_df_into_hourly(X_test_estimated_a)
X_train_estimated_b = convert_df_into_hourly(X_train_estimated_b)
X_train_observed_b = convert_df_into_hourly(X_train_observed_b)
X_test_estimated_b = convert_df_into_hourly(X_test_estimated_b)
X_train_estimated_c = convert_df_into_hourly(X_train_estimated_c)
X_train_observed_c = convert_df_into_hourly(X_train_observed_c)
X_test_estimated_c = convert_df_into_hourly(X_test_estimated_c)

## We should probably do it for the test_sets here as well


#### Removing date_calc from all estimated sets


In [413]:
X_test_estimated_a.drop("date_calc", axis=1, inplace=True)
X_train_estimated_a.drop("date_calc", axis=1, inplace=True)
X_test_estimated_b.drop("date_calc", axis=1, inplace=True)
X_train_estimated_b.drop("date_calc", axis=1, inplace=True)
X_test_estimated_c.drop("date_calc", axis=1, inplace=True)
X_train_estimated_c.drop("date_calc", axis=1, inplace=True)

#### Removing all NaN from y_targets
 - Removing all NaN from y_targets and their corresponding dates in the X_train sets, both observed and estimated
 

In [414]:
#Function that returns train_targets, observed and estimated sets left after filtering away NaN
def drop_nan_rows_in_target_and_train(y_df, observed_train_df, estimated_train_df):
    y_df = y_df.dropna(subset=['pv_measurement'])
    valid_dates = y_df['time']
    observed_train_df = observed_train_df[observed_train_df['date_forecast'].isin(valid_dates)]
    estimated_train_df = estimated_train_df[estimated_train_df['date_forecast'].isin(valid_dates)]
    return (y_df, observed_train_df, estimated_train_df)

In [415]:
#doing the NaN filtering
y_a, X_train_observed_a, X_train_estimated_a = drop_nan_rows_in_target_and_train(y_a, X_train_observed_a, X_train_estimated_a)
y_b, X_train_observed_b, X_train_estimated_b = drop_nan_rows_in_target_and_train(y_b, X_train_observed_b, X_train_estimated_b)
y_c, X_train_observed_c, X_train_estimated_c = drop_nan_rows_in_target_and_train(y_c, X_train_observed_c, X_train_estimated_c)

y_c.head()

,time,pv_measurement
5913,2019-09-04 08:00:00,137.2
5914,2019-09-04 09:00:00,0.0
5915,2019-09-04 10:00:00,0.0
5916,2019-09-04 11:00:00,0.0
5917,2019-09-04 12:00:00,0.0


In [416]:
X_train_observed_c.head()

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
5912,2019-09-04 08:00:00,6.625,1.22075,2287.250000,1320844.375,421.225006,2287.250000,0.0,278.200012,67.974998,...,130.353500,25.754999,0.0,283.725006,31.199999,49549.449219,1.250,-0.825,0.825,0.0
5913,2019-09-04 09:00:00,6.275,1.21425,2679.074951,1681730.500,508.125000,2679.074951,0.0,277.424988,76.625000,...,146.494507,30.204250,0.0,284.799988,86.250000,52309.976562,1.500,-1.500,-0.200,0.0
5914,2019-09-04 10:00:00,5.900,1.20825,2983.750000,1936799.875,561.875000,2983.750000,0.0,276.650024,112.574997,...,163.708252,33.027500,0.0,285.899994,95.149994,53978.800781,2.000,-2.000,-0.225,0.0
5915,2019-09-04 11:00:00,5.875,1.20350,3286.550049,2063526.250,578.025024,3286.550049,0.0,276.649994,195.149994,...,181.566742,33.887001,0.0,286.924988,100.000000,54510.925781,1.950,-1.950,-0.250,0.0
5916,2019-09-04 12:00:00,6.150,1.20175,3453.425049,2051320.000,555.174988,3453.425049,0.0,277.250000,243.375000,...,199.359009,32.668999,0.0,287.500000,100.000000,54284.648438,1.525,-1.500,-0.300,0.0


#### Making time features

In [417]:
#Function that creates timefeatures based on the specified coloumn
def create_time_features_based_on_coloun(df, col):   
    df['hour'] = df[col].dt.hour
    df['dayofmonth'] = df[col].dt.day
    df['dayofweek'] = df[col].dt.dayofweek
    df['quarter'] = df[col].dt.quarter
    df['month'] = df[col].dt.month
    df['year'] = df[col].dt.year
    df['dayofyear'] = df[col].dt.dayofyear
    return df


In [418]:
# adding time features

#A
X_train_estimated_a = create_time_features_based_on_coloun(X_train_estimated_a, "date_forecast")
X_train_observed_a = create_time_features_based_on_coloun(X_train_observed_a, "date_forecast")
X_test_estimated_a = create_time_features_based_on_coloun(X_test_estimated_a, "date_forecast")

#B
X_train_estimated_b = create_time_features_based_on_coloun(X_train_estimated_b, "date_forecast")
X_train_observed_b = create_time_features_based_on_coloun(X_train_observed_b, "date_forecast")
X_test_estimated_b = create_time_features_based_on_coloun(X_test_estimated_b, "date_forecast")

#C
X_train_estimated_c = create_time_features_based_on_coloun(X_train_estimated_c, "date_forecast")
X_train_observed_c = create_time_features_based_on_coloun(X_train_observed_c, "date_forecast")
X_test_estimated_c = create_time_features_based_on_coloun(X_test_estimated_c, "date_forecast")



Just showing the shapes to se if i havent f'ed up

In [419]:
# printing shapes
print("A")
print(X_train_estimated_a.shape)
print(X_train_observed_a.shape)
print(X_train_estimated_a.shape[0]+X_train_observed_a.shape[0])
print(y_a.shape)

print("B")
print(X_train_estimated_b.shape)
print(X_train_observed_b.shape)
print(X_train_estimated_b.shape[0]+X_train_observed_b.shape[0])  ## this one seems to be of by 1??????
print(y_b.shape)

print("C")
print(X_train_estimated_c.shape)
print(X_train_observed_c.shape)
print(X_train_estimated_c.shape[0]+X_train_observed_c.shape[0])
print(y_c.shape)

A
(4418, 53)
(29667, 53)
34085
(34085, 2)
B
(3625, 53)
(29218, 53)
32843
(32844, 2)
C
(2954, 53)
(23141, 53)
26095
(26095, 2)


We can see that for B there is actually on row more in the target set for some reason, finding that row and deleting it:

In [420]:
## Finding the row in y_b that does not have a match

# renaming
date_times_estimated = X_train_estimated_b['date_forecast']
date_times_observed = X_train_observed_b['date_forecast']
result_df = y_b[~y_b['time'].isin(date_times_estimated) & ~y_b['time'].isin(date_times_observed)]
print(result_df)


                 time  pv_measurement
0 2018-12-31 23:00:00             0.0


Okey so let's remove that row and verify that everything is correct:

In [421]:
# removing row
y_b = y_b[~y_b['time'].isin(result_df['time'])]

# checking that the numbers add up again
print("B")
print(X_train_estimated_b.shape)
print(X_train_observed_b.shape)
print(X_train_estimated_b.shape[0]+X_train_observed_b.shape[0])
print(y_b.shape)


B
(3625, 53)
(29218, 53)
32843
(32843, 2)


#### Removing repeated indices


#### Dividing the sets into training and validation

In [422]:
# first renaming the date_forecast columns to time  (DOES NOT WORK)
for df in list_of_all_estimated_and_observed_sets:               
    df.rename(columns={'date_forecast': 'time'}, inplace=True)

#maybe we can drop it here already actually
X_train_observed_a.head()



,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,hour,dayofmonth,dayofweek,quarter,month,year,dayofyear
0,2019-06-02 22:00:00,7.700,1.22825,1728.949951,0.000000,0.00,1728.949951,0.0,280.299988,0.000,...,-3.575,-0.500,0.0,22,2,6,2,6,2019,153
1,2019-06-02 23:00:00,7.700,1.22350,1689.824951,0.000000,0.00,1689.824951,0.0,280.299988,0.000,...,-3.350,0.275,0.0,23,2,6,2,6,2019,153
2,2019-06-03 00:00:00,7.875,1.21975,1563.224976,0.000000,0.00,1563.224976,0.0,280.649994,0.000,...,-2.950,0.750,0.0,0,3,0,2,6,2019,154
3,2019-06-03 01:00:00,8.425,1.21800,1283.425049,208.649994,0.75,1283.425049,0.0,281.674988,0.300,...,-2.600,0.875,0.0,1,3,0,2,6,2019,154
4,2019-06-03 02:00:00,8.950,1.21800,1003.500000,32468.150391,23.10,1003.500000,0.0,282.500000,11.975,...,-2.350,0.925,0.0,2,3,0,2,6,2019,154


Training data = first 75% of observed + first 75% of estimated
Validation data = last 25% of observed + last 25% of estimated

In [423]:
# Making training and validation data for A
percent_observed_train = 0.7
percent_estimated_train = 0.7

split_index_obs_a = int(len(X_train_observed_a)*percent_observed_train)
X_train_observed_a_first_75 = X_train_observed_a[:split_index_obs_a]
X_train_observed_a_last_25 = X_train_observed_a[split_index_obs_a:]

split_index_est_a = int(len(X_train_estimated_a)*percent_estimated_train)
X_train_estimated_a_first_75 = X_train_estimated_a[:split_index_est_a]
X_train_estimated_a_last_25 = X_train_estimated_a[split_index_est_a:]

X_train_a = pd.concat([X_train_observed_a_first_75, X_train_estimated_a_first_75])
y_train_a = y_a[y_a["time"].isin(X_train_a['date_forecast'])]
print(X_train_a.shape, y_train_a.shape)

X_validate_a = pd.concat([X_train_observed_a_last_25, X_train_estimated_a_last_25])
y_validate_a = y_a[y_a["time"].isin(X_validate_a['date_forecast'])]


# making training and validation for B
split_index_obs_b = int(len(X_train_observed_b)*percent_observed_train)
X_train_observed_b_first_75 = X_train_observed_b[:split_index_obs_b]
X_train_observed_b_last_25 = X_train_observed_b[split_index_obs_b:]


split_index_est_b = int(len(X_train_estimated_b)*percent_estimated_train)
X_train_estimated_b_first_75 = X_train_estimated_b[:split_index_est_b]
X_train_estimated_b_last_25 = X_train_estimated_b[split_index_est_b:]

X_train_b = pd.concat([X_train_observed_b_first_75, X_train_estimated_b_first_75])
y_train_b = y_b[y_b["time"].isin(X_train_b['date_forecast'])]
print(X_train_b.shape, y_train_b.shape)

X_validate_b = pd.concat([X_train_observed_b_last_25, X_train_estimated_b_last_25])
y_validate_b = y_b[y_b["time"].isin(X_validate_b['date_forecast'])]

# making training and validation for C
split_index_obs_c = int(len(X_train_observed_c)*percent_observed_train)
X_train_observed_c_first_75 = X_train_observed_c[:split_index_obs_c]
X_train_observed_c_last_25 = X_train_observed_c[split_index_obs_c:]

split_index_est_c = int(len(X_train_estimated_c)*percent_estimated_train)
X_train_estimated_c_first_75 = X_train_estimated_c[:split_index_est_c]
X_train_estimated_c_last_25 = X_train_estimated_c[split_index_est_c:]

X_train_c = pd.concat([X_train_observed_c_first_75, X_train_estimated_c_first_75])
y_train_c = y_c[y_c["time"].isin(X_train_c['date_forecast'])]
print(X_train_c.shape, y_train_c.shape)

X_validate_c = pd.concat([X_train_observed_c_last_25, X_train_estimated_c_last_25])
y_validate_c = y_c[y_c["time"].isin(X_validate_c['date_forecast'])]


(23858, 53) (23858, 2)
(22989, 53) (22989, 2)
(18265, 53) (18265, 2)


#### Removing repeated indexes here, think this might be the wrong place to do it, should probably to it for the concat BUT hEY

In [424]:

# def find_repeated_indexes(df, column_name, repeat_count=24):
#     """
#     Find and return the indexes of rows with a specified number of repeated values in a given column.

#     Parameters:
#     - df: DataFrame to search for repeated rows.
#     - column_name: Name of the column to check for repeated values.
#     - repeat_count: Number of repeated values required to consider a row as a match.

#     Returns:
#     - List of indexes for rows with the specified number of repeated values in the given column.
#     """
#     df = df.reset_index()
#     repeated_indexes = []
#     temp_repeated_indexes = []
#     current_value = None
#     count = 0

#     for index, row in df.iterrows():

#         value = row[column_name]

#         if value == current_value:
#             count += 1
#             temp_repeated_indexes.append(index)
#         else:
#             count = 1
#             current_value = value
#             temp_repeated_indexes = []

#         if count >= repeat_count:
#             for i in temp_repeated_indexes:
#                 if i not in repeated_indexes:
#                     repeated_indexes.append(i)

#     return repeated_indexes


# print(X_train_a.shape)
# X_train_a = X_train_a.reset_index()
# repeated_indices = find_repeated_indexes(y_train_a,"pv_measurement", 24)
# X_train_a = X_train_a.drop(index=repeated_indices)
# y_train_a = y_train_a.drop(index=repeated_indices)
# print(X_train_a.shape)

# repeated_indices = find_repeated_indexes(y_validate_a,"pv_measurement",24)
# X_validate_a = X_validate_a.reset_index()
# X_validate_a = X_validate_a.drop(index=repeated_indices)
# y_validate_a = y_train_a.drop(index=repeated_indices)

(23858, 53)


(23816, 54)


Removing time feature

In [425]:
X_train_a.drop("date_forecast", axis=1, inplace=True)
y_train_a.drop("time", axis=1, inplace=True)
X_validate_a.drop("date_forecast", axis=1, inplace=True)
y_validate_a.drop("time", axis=1, inplace=True)

X_train_b.drop("date_forecast", axis=1, inplace=True)
y_train_b.drop("time", axis=1, inplace=True)
X_validate_b.drop("date_forecast", axis=1, inplace=True)
y_validate_b.drop("time", axis=1, inplace=True)

X_train_c.drop("date_forecast", axis=1, inplace=True)
y_train_c.drop("time", axis=1, inplace=True)
X_validate_c.drop("date_forecast", axis=1, inplace=True)
y_validate_c.drop("time", axis=1, inplace=True)


C:\Users\siver\AppData\Local\Temp\ipykernel_26408\3483724575.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train_b.drop("time", axis=1, inplace=True)
C:\Users\siver\AppData\Local\Temp\ipykernel_26408\3483724575.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_validate_b.drop("time", axis=1, inplace=True)
C:\Users\siver\AppData\Local\Temp\ipykernel_26408\3483724575.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_trai

Adding location features to the sets before merge

In [426]:
X_train_a["location"] =  1 
y_train_a["location"] = 1
X_validate_a["location"] = 1
y_validate_a["location"] = 1

X_train_b["location"] = 2
y_train_b["location"] = 2
X_validate_b["location"] = 2
y_validate_b["location"] = 2

X_train_c["location"] = 3
y_train_c["location"] = 3
X_validate_c["location"] = 3
y_validate_c["location"] = 3


C:\Users\siver\AppData\Local\Temp\ipykernel_26408\816523439.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train_b["location"] = 2
C:\Users\siver\AppData\Local\Temp\ipykernel_26408\816523439.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_validate_b["location"] = 2
C:\Users\siver\AppData\Local\Temp\ipykernel_26408\816523439.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

Merging tranining data:


In [427]:
X_train = pd.concat([X_train_a, X_train_b, X_train_c])
y_train = pd.concat([y_train_a, y_train_b, y_train_c])

X_validate = pd.concat([X_validate_a, X_validate_b, X_validate_c])

print(X_train.shape)
print(y_train.shape)

(65070, 54)
(65070, 2)


One-hot encoding

In [428]:
# one hot encoding and dropping location feature   some fuckery here
# one_hot = pd.get_dummies(X_train["location"]).astype(int)
# X_train = X_train.drop("location", axis=1)
# X_train = pd.merge(X_train, one_hot, left_index=True, right_index=True)


# one_hot = pd.get_dummies(X_validate["location"]).astype(int)
# X_validate = pd.merge(X_validate, one_hot, left_index=True, right_index=True)


# print(X_train.shape)
# print(y_train.shape)

Let's test an XG boost only on A

In [429]:
model = xgb.XGBRegressor(
    max_depth=7,
    colsample_bytree=0.8,
    eta=0.1,
    n_estimators=90,
    reg_alpha=0.01,
    reg_lambda=0.01,
    enable_categorical=True
)

model.fit(X_train, y_train["pv_measurement"])

# X_validate_a_loc = X_validate[X_validate_a["location"]=="A"]
# X_validate_a_loc = X_validate_a_loc.drop("location", axis=1)

prediction_a = model.predict(X_validate_a)
prediction_b = model.predict(X_validate_b)
prediction_c = model.predict(X_validate_c)

c:\Users\siver\NTNU Høst 2023\Machine Learning\Group Project\tdt4173-forecasting\.venv\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\siver\NTNU Høst 2023\Machine Learning\Group Project\tdt4173-forecasting\.venv\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\siver\NTNU Høst 2023\Machine Learning\Group Project\tdt4173-forecasting\.venv\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\siver\NTNU Høst 2023\Machine Learning\Group Project\tdt4173-forecasting\.venv\l

c:\Users\siver\NTNU Høst 2023\Machine Learning\Group Project\tdt4173-forecasting\.venv\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\siver\NTNU Høst 2023\Machine Learning\Group Project\tdt4173-forecasting\.venv\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\siver\NTNU Høst 2023\Machine Learning\Group Project\tdt4173-forecasting\.venv\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\siver\NTNU Høst 2023\Machine Learning\Group Project\tdt4173-forecasting\.venv\l

ValueError: feature_names mismatch: ['index', 'absolute_humidity_2m:gm3', 'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J', 'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx', 'dew_point_2m:K', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J', 'effective_cloud_cover:p', 'elevation:m', 'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx', 'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa', 'snow_density:kgm3', 'snow_depth:cm', 'snow_drift:idx', 'snow_melt_10min:mm', 'snow_water:kgm2', 'sun_azimuth:d', 'sun_elevation:d', 'super_cooled_liquid_water:kgm2', 't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m', 'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:ms', 'hour', 'dayofmonth', 'da

In [ ]:
print(
    "MAE: ",
    round(
        mean_absolute_error(
            y_true=y_validate_a["pv_measurement"],
            y_pred=prediction_a,
        ),
        3,
    ),
)

ValueError: Found input variables with inconsistent numbers of samples: [23816, 10227]

In [ ]:
print(
    "MAE: ",
    round(
        mean_absolute_error(
            y_true=y_validate_b["pv_measurement"],
            y_pred=prediction_b,
        ),
        3,
    ),
)

MAE:  56.58


In [ ]:
print(
    "MAE: ",
    round(
        mean_absolute_error(
            y_true=y_validate_c["pv_measurement"],
            y_pred=prediction_c,
        ),
        3,
    ),
)

MAE:  25.492
